<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/master/api_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install xmltodict

In [3]:
import pandas as pd
import re
import sys
import requests
import json
import pprint
from bs4 import BeautifulSoup
from urllib.request import urlopen
import xmltodict
from tqdm import tqdm
import random
import pprint

In [4]:
url = 'http://www.law.go.kr/DRF/lawSearch.do?OC=seokhee051&target=elaw&type=XML'
# url 불러오기
response = requests.get(url)
# XML -> 딕셔너리 변환후 파싱
contents = response.text
dict_contents = xmltodict.parse(contents)
totalCnt = int(dict_contents['LawSearch']['totalCnt'])

In [5]:
totalCnt

2609

In [8]:
rows = []
not_match_jo = []
changed_num = 0
changed = []
eng_cnt = 0
kor_cnt = 0
total_page = int(totalCnt/20)+1
for p in tqdm(range(120,total_page)):
  url = 'http://www.law.go.kr/DRF/lawSearch.do?OC=seokhee051&target=elaw&type=XML' \
        + '&page=' + str(p)
  # url 불러오기
  response = requests.get(url)
  # XML -> 딕셔너리 변환후 파싱
  contents = response.text
  dict_contents = xmltodict.parse(contents)
  try:
    for content in dict_contents['LawSearch']['law']:
      name_kor = content['법령명한글']
      name_eng = content['법령명영문']
      mst = content['법령일련번호']

      eng_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=elaw' \
                + '&MST=' + str(mst)
                # + '&ID=' + str(법령ID)
      eng_response = requests.get(eng_url)
      eng_contents = eng_response.text
      eng_dict_contents = xmltodict.parse(eng_contents)

      kor_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=law' \
                + '&MST=' + str(mst)
                # + '&ID=' + str(법령ID)
      kor_response = requests.get(kor_url)
      kor_contents = kor_response.text
      kor_dict_contents = xmltodict.parse(kor_contents)

      # # 영어조문 개수와 한글 조문 개수 맞지 않다면 건너뛰기
      # if len(eng_dict_contents['Law']['JoSection']['Jo']) != len(kor_dict_contents['법령']['조문']['조문단위']):
      #     not_match_jo += 1
      #     print(f'영한 조문 개수 맞지 않는 법령 개수', not_match_jo, kor_dict_contents['법령']['기본정보']['법령명_한글'])
      #     continue
      # eng_cnt += len(eng_dict_contents['Law']['JoSection']['Jo'])
      # kor_cnt += len(kor_dict_contents['법령']['조문']['조문단위'])
      min_num_jo = min(len(eng_dict_contents['Law']['JoSection']['Jo']), len(kor_dict_contents['법령']['조문']['조문단위']))
      temp_cnt = 0
      print('법령명:', name_kor, ', 조문 개수:', min_num_jo, ', 페이지:', p, ', 몇번째?:', content['@id'])
      try:
        # for eng_jo, kor_jo in zip(eng_dict_contents['Law']['JoSection']['Jo'],kor_dict_contents['법령']['조문']['조문단위']):
        for ind in range(min_num_jo):
          eng_jo, kor_jo = eng_dict_contents['Law']['JoSection']['Jo'][ind], kor_dict_contents['법령']['조문']['조문단위'][ind]

          if kor_jo['조문여부']!='조문':
            continue
          # if kor_jo.get('조문가지번호'):
          #   continue

          kor_total = 0
          if kor_jo.get('항'): # 항내용이 있으면 list 바로 호내용이면 dict 타입이네
            if type(kor_jo['항']) == dict:
              kor_total += len(kor_jo['항']['호'])
              for ho in kor_jo['항']['호']:
                if ho.get('목'):
                  kor_total += len(ho['목'])
              kor_total += 1 # 호부터 시작하면 영어에서는 또 \n이 하나 늘어남
            else:
              kor_total += len(kor_jo['항'])
              for hang in kor_jo['항']:
                if hang.get('호'):
                  kor_total += len(hang['호'])
                  for ho in hang['호']:
                    if ho.get('목'):
                      kor_total += len(ho['목'])
          else:
            kor_total = 1

          if kor_jo.get('조문참고자료'):
            if '\n\t\t\n\t\t' in kor_jo['조문참고자료']:
              kor_total += 2
              kor_jo['조문참고자료'] = kor_jo['조문참고자료'].replace('\n\t\t\n\t\t', '\n')
            else:
              kor_total += 1

          if (eng_jo['joCts'].count('\n')+1) != kor_total:
            not_match_jo.append([name_kor, kor_jo['조문제목']])
            continue

          if eng_jo['joYn']!= 'Y':
            changed_num += 1
            changed.append([eng_jo, kor_jo])
            print('안바뀐 법령 개수:', changed_num)
            continue
          # if eng_jo['joBrNo']!='00':
          #   continue

          eng_text = eng_jo['joCts']

          jo_txt = ''
          hang_txt = ''
          ho_txt = ''
          mok_txt = ''

          jo_txt += kor_jo['조문내용']
          if kor_jo.get('항'): # 항내용이 있으면 list 바로 호내용이면 dict 타입이네
            if type(kor_jo['항']) == dict: # 바로 호내용일 때
              for ho in kor_jo['항']['호']:
                ho_txt += '\n' + ho['호내용']
                if ho.get('목'):
                  for mok in ho['목']:
                    mok_txt += '\n' + mok['목내용']
                  ho_txt += mok_txt
              hang_txt += ho_txt
            else:
              for hang in kor_jo['항']:
                if hang_txt == '':
                  hang_txt += hang['항내용'] # 영어의 경우 맨 첫항과 조문 내용 사이에 \n으로 분리가 안돼있음
                else:
                  hang_txt += '\n' + hang['항내용']

                if hang.get('호'):
                  for ho in hang['호']:
                    ho_txt += '\n' + ho['호내용']
                    if ho.get('목'):
                      for mok in ho['목']:
                        mok_txt += '\n' + mok['목내용']
                      ho_txt += mok_txt
                  hang_txt += ho_txt
              jo_txt += hang_txt
          if kor_jo.get('조문참고자료'):
            eng_text = '\n'.join([i.strip() for i in eng_text.split('\n')[:-1]]) # 조문참고자료 부분 삭제하기
          kor_text = jo_txt
          rows.append({'name_kor':name_kor, 'name_eng':name_eng, 'MST':mst, 'eng':eng_text, 'kor':kor_text})
      except Exception as e:
        print('eng_kor_contents 예외 발생:', e)
  except Exception as e:
    print('dict_contents 예외 발생:', e)
print(len(not_match_jo))
df = pd.DataFrame(rows)

  0%|          | 0/11 [00:00<?, ?it/s]

법령명: 한국국방연구원법 , 조문 개수: 23 , 페이지: 120 , 몇번째?: 2381
법령명: 한국국제교류재단법 , 조문 개수: 28 , 페이지: 120 , 몇번째?: 2382
법령명: 한국국제보건의료재단법 , 조문 개수: 31 , 페이지: 120 , 몇번째?: 2383
법령명: 한국국제보건의료재단법 시행령 , 조문 개수: 16 , 페이지: 120 , 몇번째?: 2384
법령명: 한국국제협력단법 , 조문 개수: 35 , 페이지: 120 , 몇번째?: 2385
법령명: 한국농어촌공사 및 농지관리기금법 , 조문 개수: 67 , 페이지: 120 , 몇번째?: 2386
법령명: 한국도로공사법 , 조문 개수: 28 , 페이지: 120 , 몇번째?: 2387
법령명: 한국보건의료인국가시험원법 , 조문 개수: 26 , 페이지: 120 , 몇번째?: 2388
법령명: 한국보훈복지의료공단법 , 조문 개수: 35 , 페이지: 120 , 몇번째?: 2389
eng_kor_contents 예외 발생: '조문제목'
법령명: 한국부동산원법 , 조문 개수: 21 , 페이지: 120 , 몇번째?: 2390
법령명: 한국사학진흥재단법 , 조문 개수: 36 , 페이지: 120 , 몇번째?: 2391
법령명: 한국산업은행법 , 조문 개수: 54 , 페이지: 120 , 몇번째?: 2392
법령명: 한국산업은행법 시행령 , 조문 개수: 49 , 페이지: 120 , 몇번째?: 2393
법령명: 한국산업은행법시행규칙 , 조문 개수: 7 , 페이지: 120 , 몇번째?: 2394
eng_kor_contents 예외 발생: 0
법령명: 한국수자원공사법 , 조문 개수: 48 , 페이지: 120 , 몇번째?: 2395
eng_kor_contents 예외 발생: '조문제목'
법령명: 한국수자원공사법 시행령 , 조문 개수: 39 , 페이지: 120 , 몇번째?: 2396
법령명: 한국수출입은행법 , 조문 개수: 57 , 페이지: 120 , 몇번째?: 2397
법령명: 한국수출입은행법 시행령 , 조문 개수: 

  9%|▉         | 1/11 [02:27<24:33, 147.32s/it]

법령명: 한국수화언어법 시행령 , 조문 개수: 14 , 페이지: 120 , 몇번째?: 2400
법령명: 한국여성개발원법 , 조문 개수: 25 , 페이지: 121 , 몇번째?: 2401
법령명: 한국연구재단법 , 조문 개수: 24 , 페이지: 121 , 몇번째?: 2402
법령명: 한국연구재단법 시행령 , 조문 개수: 9 , 페이지: 121 , 몇번째?: 2403
법령명: 한국원자력안전기술원법 , 조문 개수: 24 , 페이지: 121 , 몇번째?: 2404
법령명: 한국원자력안전기술원법 시행령 , 조문 개수: 7 , 페이지: 121 , 몇번째?: 2405
법령명: 한국은행법 , 조문 개수: 137 , 페이지: 121 , 몇번째?: 2406
법령명: 한국은행 통화안정증권법 , 조문 개수: 8 , 페이지: 121 , 몇번째?: 2407
법령명: 한국인 원자폭탄 피해자 지원을 위한 특별법 , 조문 개수: 19 , 페이지: 121 , 몇번째?: 2408
법령명: 한국인 원자폭탄 피해자 지원을 위한 특별법 시행령 , 조문 개수: 13 , 페이지: 121 , 몇번째?: 2409
법령명: 한국자산관리공사 설립 등에 관한 법률 , 조문 개수: 69 , 페이지: 121 , 몇번째?: 2410
법령명: 한국장학재단 설립 등에 관한 법률 , 조문 개수: 67 , 페이지: 121 , 몇번째?: 2411
법령명: 한국장학재단 설립 등에 관한 법률 시행령 , 조문 개수: 41 , 페이지: 121 , 몇번째?: 2412
법령명: 한국전통문화대학교 설치법 , 조문 개수: 21 , 페이지: 121 , 몇번째?: 2413
법령명: 한국정책금융공사법 , 조문 개수: 45 , 페이지: 121 , 몇번째?: 2414
법령명: 한국정책금융공사법 시행령 , 조문 개수: 40 , 페이지: 121 , 몇번째?: 2415
법령명: 한국주택금융공사법 , 조문 개수: 96 , 페이지: 121 , 몇번째?: 2416
eng_kor_contents 예외 발생: '조문제목'
법령명: 한국주택금융공사법 시행령 , 조문

 18%|█▊        | 2/11 [04:47<21:28, 143.20s/it]

...

dict_contents 예외 발생: 'NoneType' object is not subscriptable
법령명: 화재의 예방 및 안전관리에 관한 법률 , 조문 개수: 56 , 페이지: 129 , 몇번째?: 2561
법령명: 화재의 예방 및 안전관리에 관한 법률 시행령 , 조문 개수: 58 , 페이지: 129 , 몇번째?: 2562
법령명: 화학무기ㆍ생물무기의 금지와 특정화학물질ㆍ생물작용제 등의 제조ㆍ수출입 규제 등에 관한 법률 , 조문 개수: 46 , 페이지: 129 , 몇번째?: 2563
법령명: 화학물질관리법 , 조문 개수: 88 , 페이지: 129 , 몇번째?: 2564
법령명: 화학물질관리법 시행령 , 조문 개수: 29 , 페이지: 129 , 몇번째?: 2565
법령명: 화학물질관리법 시행규칙 , 조문 개수: 94 , 페이지: 129 , 몇번째?: 2566
법령명: 화학물질의 등록 및 평가 등에 관한 법률 , 조문 개수: 69 , 페이지: 129 , 몇번째?: 2567
법령명: 화학물질의 등록 및 평가 등에 관한 법률 시행령 , 조문 개수: 51 , 페이지: 129 , 몇번째?: 2568
법령명: 화학물질의 등록 및 평가 등에 관한 법률 시행규칙 , 조문 개수: 72 , 페이지: 129 , 몇번째?: 2569
법령명: 환경개선비용 부담법 , 조문 개수: 24 , 페이지: 129 , 몇번째?: 2570
법령명: 환경개선비용 부담법 시행령 , 조문 개수: 32 , 페이지: 129 , 몇번째?: 2571
법령명: 환경개선특별회계법 , 조문 개수: 11 , 페이지: 129 , 몇번째?: 2572
법령명: 환경교육진흥법 , 조문 개수: 20 , 페이지: 129 , 몇번째?: 2573
법령명: 환경기술 및 환경산업 지원법 , 조문 개수: 78 , 페이지: 129 , 몇번째?: 2574
eng_kor_contents 예외 발생: '조문제목'
법령명: 환경기술 및 환경산업 지원법 시행령 , 조문 개수: 74 , 페이지: 129 , 몇번째?: 2575
법령명: 환경

 91%|█████████ | 10/11 [23:57<02:17, 137.38s/it]

법령명: 환경분야 시험ㆍ검사 등에 관한 법률 , 조문 개수: 48 , 페이지: 129 , 몇번째?: 2580
법령명: 환경분야 시험ㆍ검사 등에 관한 법률 시행령 , 조문 개수: 24 , 페이지: 130 , 몇번째?: 2581
법령명: 환경분쟁 조정법 , 조문 개수: 88 , 페이지: 130 , 몇번째?: 2582
법령명: 환경영향평가법 , 조문 개수: 106 , 페이지: 130 , 몇번째?: 2583
법령명: 환경영향평가법 시행령 , 조문 개수: 115 , 페이지: 130 , 몇번째?: 2584
법령명: 환경오염시설의 통합관리에 관한 법률 , 조문 개수: 65 , 페이지: 130 , 몇번째?: 2585
법령명: 환경오염시설의 통합관리에 관한 법률 시행령 , 조문 개수: 48 , 페이지: 130 , 몇번째?: 2586
법령명: 환경오염피해 배상책임 및 구제에 관한 법률 , 조문 개수: 57 , 페이지: 130 , 몇번째?: 2587
법령명: 환경오염피해 배상책임 및 구제에 관한 법률 시행령 , 조문 개수: 41 , 페이지: 130 , 몇번째?: 2588
법령명: 환경정책기본법 , 조문 개수: 80 , 페이지: 130 , 몇번째?: 2589
법령명: 환경정책기본법 시행령 , 조문 개수: 38 , 페이지: 130 , 몇번째?: 2590
법령명: 환경친화적 산업구조로의 전환촉진에 관한 법률 , 조문 개수: 59 , 페이지: 130 , 몇번째?: 2591
eng_kor_contents 예외 발생: '조문제목'
법령명: 환경친화적 산업구조로의 전환촉진에 관한 법률 시행령 , 조문 개수: 43 , 페이지: 130 , 몇번째?: 2592
eng_kor_contents 예외 발생: '조문제목'
법령명: 환경친화적 자동차의 개발 및 보급 촉진에 관한 법률 , 조문 개수: 23 , 페이지: 130 , 몇번째?: 2593
법령명: 환경친화적 자동차의 개발 및 보급 촉진에 관한 법률 시행령 , 조문 개수: 33 , 페이지: 130 , 몇번째?: 2594
법령명: 환자안전법 , 조문 

100%|██████████| 11/11 [26:33<00:00, 144.88s/it]

법령명: 후견등기에 관한 법률 , 조문 개수: 52 , 페이지: 130 , 몇번째?: 2600
966


In [9]:
df.to_csv('/content/drive/MyDrive/KB/KB_trans/120-.csv', index=False)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
url = 'http://www.law.go.kr/DRF/lawSearch.do?OC=seokhee051&target=elaw&type=XML' \
        + '&page=1'
# url 불러오기
response = requests.get(url)
# XML -> 딕셔너리 변환후 파싱
contents = response.text
dict_contents = xmltodict.parse(contents)

content = dict_contents['LawSearch']['law'][5]

name_kor = content['법령명한글']
name_eng = content['법령명영문']
mst = content['법령일련번호']
print(name_kor)
eng_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=elaw' \
          + '&MST=' + str(mst)
          # + '&ID=' + str(법령ID)
eng_response = requests.get(eng_url)
eng_contents = eng_response.text
eng_dict_contents = xmltodict.parse(eng_contents)

kor_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=law' \
          + '&MST=' + str(mst)
          # + '&ID=' + str(법령ID)
kor_response = requests.get(kor_url)
kor_contents = kor_response.text
kor_dict_contents = xmltodict.parse(kor_contents)

2018 평창 동계올림픽대회 및 동계패럴림픽대회 지원 등에 관한 특별법


KeyboardInterrupt: ignored

In [ ]:
count = 0
for temp in kor_dict_contents['법령']['조문']['조문단위']:
    if temp.get('조문제목'):
        if temp['조문제목'] == '기념화폐의 판매':
            print(count)
    count += 1

17


In [ ]:
eng_dict_contents['Law']['JoSection']['Jo'][6]

{'@No': '7',
 'chapNo': '00000200000000000000',
 'joNo': '0005',
 'joBrNo': '00',
 'joYn': 'Y',
 'joTtl': 'Establishment of Organizing Committee',
 'joCts': 'Article 5 (Establishment of Organizing Committee) (1) In order to conduct the following projects and activities for the preparation and successful operation of the Games, the PyeongChang Organizing Committee for the 2018 Olympic and Paralympic Winter Games (hereinafter referred to as "the Organizing Committee") shall be established upon authorization from the Minister of Culture, Sports and Tourism: <Amended by Act No. 14198, May 29, 2016>\n  1. Establishment of a comprehensive plan for the Games, and establishment and  implementation of detailed operational plans;\n  2. Establishment and implementation of a plan to establish and use Games-related facilities;\n  3. Cooperation with international sports organizations promoting the Games;\n  4. Other projects necessary for the smooth preparation and operation of the Games.\n  (2) Th

In [ ]:
kor_dict_contents['법령']['조문']['조문단위'][6]

{'@조문키': '0005001',
 '조문번호': '5',
 '조문여부': '조문',
 '조문제목': '조직위원회의 설립',
 '조문시행일자': '20190401',
 '조문이동이전': None,
 '조문이동이후': None,
 '조문변경여부': 'N',
 '조문내용': '제5조(조직위원회의 설립)',
 '항': [{'항번호': '①',
   '항내용': '① 대회의 준비 및 성공적인 개최를 위한 다음 각 호의 사업과 활동을 하기 위하여 문화체육관광부장관의 인가를 받아 2018 평창 동계올림픽대회 및 동계패럴림픽대회 조직위원회(이하 "조직위원회"라 한다)를 설립한다. <개정 2016.5.29>',
   '호': [{'호번호': '1.', '호내용': '1. 대회 종합계획 수립 및 세부 운영계획의 수립ㆍ시행'},
    {'호번호': '2.', '호내용': '2. 대회관련시설의 설치ㆍ이용에 관한 계획의 수립ㆍ시행'},
    {'호번호': '3.', '호내용': '3. 대회를 주최하는 국제스포츠기구와의 협력'},
    {'호번호': '4.', '호내용': '4. 그 밖에 대회의 원활한 준비 및 운영을 위하여 필요한 사업'}]},
  {'항번호': '②', '항내용': '② 조직위원회는 법인으로 한다.'},
  {'항번호': '③',
   '항내용': '③ 조직위원회에 위원장, 부위원장, 집행위원 및 감사를 두고 이를 임원으로 하며, 최고의결기구인 위원총회 및 업무의 효율적인 추진을 위한 집행위원회를 둔다.'},
  {'항번호': '④',
   '항내용': '④ 제3항에 따른 임원의 정원, 임기 및 선출 방법, 위원총회와 집행위원회의 구성 및 기능 등에 관한 사항은 정관으로 정한다.'},
  {'항번호': '⑤',
   '항내용': '⑤ 조직위원회에 관하여 이 법에서 규정한 것 외에는 「민법」 중 재단법인에 관한 규정을 준용한다.'}],
 '조문참고자료': '[법률 제11226호(2012. 1. 26.) 제5조의 개정규정은 같은 법 부칙 제2조의 규정에 의하여

In [ ]:
# 영한 조문 개수 맞지 않는 법령 개수 2 간접투자자산 운용업법 시행령 17
# 영한 조문 개수 맞지 않는 법령 개수 3 간척지의 농어업적 이용 및 관리에 관한 법률 9
# 영한 조문 개수 맞지 않는 법령 개수 4 개발이익 환수에 관한 법률 14
# 영한 조문 개수 맞지 않는 법령 개수 5 개별소비세법 18

In [ ]:
kor_jo = kor_dict_contents['법령']['조문']['조문단위'][43]
eng_jo = eng_dict_contents['Law']['JoSection']['Jo'][43]
kor_total = 0
if kor_jo.get('항'): # 항내용이 있으면 list 바로 호내용이면 dict 타입이네
  if type(kor_jo['항']) == dict:
    kor_total += len(kor_jo['항']['호'])
    for ho in kor_jo['항']['호']:
      if ho.get('목'):
        kor_total += len(ho['목'])
    kor_total += 1
  else:
    kor_total += len(kor_jo['항'])
    for hang in kor_jo['항']:
      if hang.get('호'):
        kor_total += len(hang['호'])
        for ho in hang['호']:
          if ho.get('목'):
            kor_total += len(ho['목'])
else:
  kor_total = 1
print(kor_total)

if kor_jo.get('조문참고자료'):
  kor_total += (1 + kor_jo['조문참고자료'].count('\n'))
print(kor_total)
if (eng_jo['joCts'].count('\n')+1) != kor_total:
  not_match_jo += 1
  print(not_match_jo)

6
9
4
